# Anatomy of a `tf.Graph`

A `tf.Graph` contains a set of `tf.Operation` objects, which represent units of computation; and `tf.Tensor` objects, which represent the units of data that flow between operations. We can see a `tf.Graph` (also in the *pythonic* way) as a context for the creation and the organization of such objects. All the objects created in TF belongs to a `tf.Graph`. 

First of all, we have to say that even if not  specified explicitly, they are bound to a default one:

In [1]:
import tensorflow as tf

x = tf.Variable(23, 'x')
assert x.graph == tf.get_default_graph()
print('the default graph id is: %s' % str(id(tf.get_default_graph())))

the default graph id is: 140266256133328


## Graphs as contexts
If we want to bind variables (and ops and tensors) to a particular graph, we have to us it as a context and wrap the variable creation in its scope through the `with` construct and the `tf.Graph.as_default()` method: this will made the given `tf.Graph` instance the default graph in that scope:

In [2]:
graph = tf.Graph()
print('the new graph id is: %s' % str(id(graph)))
assert graph != tf.get_default_graph()
with graph.as_default() as g:
    y = tf.Variable(9, 'x')
    assert g == tf.get_default_graph()
    assert g == y.graph
    print('the default graph in this context scope is: %s' % str(id(tf.get_default_graph())))
print('again outside the scope, the default graph id is: %s' % str(id(tf.get_default_graph())))

the new graph id is: 140265636620048
the default graph in this context scope is: 140265636620048
again outside the scope, the default graph id is: 140266256133328


## Graph Collections
One interesting feature of the `tf.Graph` is that it holds some *collections* that are particularly handy when you have to manage variables, activations, summaries (see later) and queue runners (see later, as well). In particular, when you create a variable, it is already stored into proper collections of the graph so that it can be accessed later very easily.---this holds not only for variables, of course.  
  
You can access such collections with their keys, which are stored in the `tf.GraphKeys` class:

In [3]:
print('Here are all the keys to access collections in `tf.Graph`:')
for attr in dir(tf.GraphKeys):
    if not attr.startswith('_'):
        print '  ' + attr

Here are all the keys to access collections in `tf.Graph`:
  ACTIVATIONS
  ASSET_FILEPATHS
  BIASES
  CONCATENATED_VARIABLES
  COND_CONTEXT
  EVAL_STEP
  GLOBAL_STEP
  GLOBAL_VARIABLES
  INIT_OP
  LOCAL_INIT_OP
  LOCAL_RESOURCES
  LOCAL_VARIABLES
  LOSSES
  MODEL_VARIABLES
  MOVING_AVERAGE_VARIABLES
  QUEUE_RUNNERS
  READY_FOR_LOCAL_INIT_OP
  READY_OP
  REGULARIZATION_LOSSES
  RESOURCES
  SAVEABLE_OBJECTS
  SAVERS
  SUMMARIES
  SUMMARY_OP
  TABLE_INITIALIZERS
  TRAINABLE_RESOURCE_VARIABLES
  TRAINABLE_VARIABLES
  TRAIN_OP
  UPDATE_OPS
  VARIABLES
  WEIGHTS
  WHILE_CONTEXT


Such collections are intended to have a **clear semantics**: you won't be looking for trainable variables in the `QUEUE_RUNNERS` collections (and viceversa). Such semantics is used implicitly by many functions and ops in the default TF library so that when an object is created, it is automatically added to the proper collection. As an example, when we create a `tf.Variable`, it is added to the `GLOBAL_VARIABLES` collection and if we make it trainable, it is added to the `TRAINABLE_VARIABLES` as well.

In [4]:
with tf.Graph().as_default() as graph:
    x = tf.Variable(23, name='x')
    y = tf.Variable(9, name='y', trainable=False)
    
print('let\'s inspect the graph %s' % str(id(graph)))
print('The %s are:' % tf.GraphKeys.GLOBAL_VARIABLES)
variables = graph.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
for var in variables:
    print var.name
print

print('The %s are:' % tf.GraphKeys.TRAINABLE_VARIABLES)
variables = graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
for var in variables:
    print var.name
print


let's inspect the graph 140265636622096
The variables are:
x:0
y:0

The trainable_variables are:
x:0

